Goal of this notebook:
 - Determine whether subgraphing can be done in Gremlin-Python
 - If so, implement subgraphing based on time and perform path queries on it. Benchmark those queries.

In [29]:
# Jupyter notebook needs this or else it will crash
from datetime import datetime
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver import client
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')
from gremlin_python.process.strategies import SubgraphStrategy

# Instantiate a Gremlin Graph
graph = Graph()

# Instantiate Gremlin client
gremlin_client = client.Client('ws://localhost:8182/gremlin', 'g')

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [30]:
# Test.

time = 2

sg = g.withStrategies(SubgraphStrategy(edges=__.and_(
    __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
    __.or_(
        __.hasNot('end'),           # end time doesn't have to exist 
        __.has('end', P.gt(time))  # OR end time must be greater than <time>
    )
)))

print(sg.V().count().next())
print(sg.E().count().next())

12297
13312
